In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_vertical_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,39196.00,0.68,0.20,0.13,0.01,0.00,-0.00,-0.00
3,0.68,8670.46,-0.09,0.05,-0.00,0.00,-0.00,0.00
4,0.20,-0.09,1583.40,0.01,0.00,-0.00,0.00,-0.00
5,0.13,0.05,0.01,257.67,0.00,0.00,0.00,-0.00
6,0.01,-0.00,0.00,0.00,36.73,0.00,0.00,0.00
7,0.00,0.00,-0.00,0.00,0.00,4.83,-0.00,0.00
8,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.62,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.07


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00003,0.00004,0.00000,0.00000,-0.00000,-0.00001
3,0.00004,1.00000,-0.00002,0.00003,-0.00000,0.00001,-0.00003,0.00003
4,0.00003,-0.00002,1.00000,0.00002,0.00001,-0.00005,0.00004,-0.00003
5,0.00004,0.00003,0.00002,1.00000,0.00001,0.00002,0.00001,-0.00002
6,0.00000,-0.00000,0.00001,0.00001,1.00000,0.00002,0.00002,0.00002
7,0.00000,0.00001,-0.00005,0.00002,0.00002,1.00000,-0.00003,0.00001
8,-0.00000,-0.00003,0.00004,0.00001,0.00002,-0.00003,1.00000,0.00002
9,-0.00001,0.00003,-0.00003,-0.00002,0.00002,0.00001,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.0340934257547185

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.92247024e+08 9.20491494e+06 2.98503352e+05 7.58934653e+03
 1.44649176e+02 2.22987012e+00 2.72775634e-02 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999434,-0.033141,-0.005723,-0.000927,-0.000133,-0.000017,-0.000002,-2.563485e-07
1,0.032938,0.998943,-0.031659,-0.004949,-0.000706,-0.000092,-0.000012,-1.214017e-06
2,0.006725,0.031266,0.998914,-0.033546,-0.004633,-0.000612,-0.000077,-8.760962e-06
3,0.001308,0.005930,0.033174,0.998754,-0.036477,-0.004679,-0.000595,-6.634917e-05
4,0.000233,0.001055,0.005778,0.036050,0.998317,-0.044695,-0.005560,-6.157858e-04
5,0.000041,0.000184,0.001012,0.006205,0.044037,0.996893,-0.064632,-6.991624e-03
6,0.000008,0.000035,0.000191,0.001174,0.008239,0.063153,0.991793,-1.108570e-01
7,0.000002,0.000008,0.000043,0.000263,0.001845,0.014030,0.110174,9.938116e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0005661268466394809,
 0.0010566614695994847,
 0.0010855889049414813,
 0.001245885084381393,
 0.0016829629120982403,
 0.0031073521805335957,
 0.008207105329949504,
 0.006188423985521885]